In [1]:
dataName = 'chile' # The name of the data file. Now it is just for different language.
num_of_representatives = 8 # The number of representative narrative you want to show for each topics.
num_of_topics = 50 # The number of topics you want to generate from the data.
# For output
import os
import pathlib
import csv


import re
import numpy as np
import pandas as pd
from pprint import pprint
import nltk; 
nltk.download('stopwords')

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


e:\coding\environment\python3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# Prepare the output folder, according to the input data's language type
outputPath = 'output/' + dataName
if not os.path.exists(outputPath):
    os.mkdir(outputPath)
outputPath = outputPath + '/'

In [3]:
# Prepare the input data's path, according to the input data's language type
inputPath = 'input/' + dataName + '/'

In [4]:
%matplotlib inline

In [5]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [6]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [8]:
from nltk.corpus import stopwords
stop_words = stopwords.words('English')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [9]:
df = pd.read_json(inputPath + dataName + '_data.json')
print(df.Narrative.unique())
df.head()

['cuando fallecio mi cu帽ado'
 'cuando me  cambie de trabajo ,pensando que era  lo mejor y no fue  as铆 ,antes de  eso estaba muy bien trabajando,deseaba  que  eso jamas hubiese pasado es decir  que  yo no hubiera  tomado esa mala desicion'
 'Cuando naci贸 mi hijo'
 'CUANDO MI PAREJA ME DICE QUE LO DESPIDIERON DE SU TRABAJO / FUE HACE 1 A脩O APROX / YO ESTABA EN MI CASA Y EL ME LO COMUNICO TELEFONICAMENTE / ESTABA COCINANDO ANTES DE... / DESEABA LLORAR'
 'Mi Gatito desapareci贸 por 2 semanas y lo encontramos, ocurri贸 el mes de abril, estaba en la casa con mis padres, estaba tratando de estudiar pero luego no me pude concentrar ni dormir esper茅 hasta que lo trajeran a la casa.'
 'bueno empez贸 cuando tenia que disertar frente a un grupo grande de personas a las cuales / tenia  que relatar todo lo que hab铆a recopilado a lo largo de los a帽os pero eso me pon铆a nervioso / porque no sabia si lo que dec铆a estaba bien o mal as铆 que una vez dicho la informaci贸n me asombre  / porque hab铆a logrado supe

 'Terremoto 27 de febrero del 2010 si no me equivoco']


,,Age,AgeNow,AgeThen,BirthPlace,Body_X1,Body_X2,Body_X3,Body_X4,Body_X5,...,X.8,X.9,code,country,gc,number,opp,rid,term,translation
0,1475,31,14,3,9,20.0,319.0,286.0,92.0,92.0,...,NA,NA,1,Chile,1,1475,Qual961-0923Countries,NA,NA,When my brother-in-law passed away
1,1486,53,36,6,9,288.0,147.0,78.0,362.0,276.0,...,NA,NA,1,Chile,1,1486,Qual961-0923Countries,NA,NA,"When I changed jobs, I thought it was for the ..."
2,1488,24,6,3,9,95.0,112.0,175.0,12.0,294.0,...,NA,NA,1,Chile,1,1488,Qual961-0923Countries,NA,NA,When my son was born
3,1489,29,12,3,9,85.0,92.0,359.0,218.0,285.0,...,NA,NA,1,Chile,1,1489,Qual961-0923Countries,NA,NA,When my partner told me that they got fired fr...
4,1491,21,3,3,9,264.0,287.0,88.0,112.0,169.0,...,NA,NA,1,Chile,1,1491,Qual961-0923Countries,NA,NA,My kitten disappeared for two weeks and we fin...


In [10]:
# Convert to list
if df.X.values is 'English':
    data = df.Narrative.values.tolist()
else:
    data = df.translation.values.tolist()
# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['When my brother-in-law passed away']


In [11]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['when', 'my', 'brother', 'in', 'law', 'passed', 'away']]


In [12]:
# Build bigram and trigram
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print(trigram_mod[bigram_mod[data_words[0]]])

e:\coding\environment\python3\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['when', 'my', 'brother', 'in', 'law', 'passed', 'away']


In [13]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [14]:
#remove stop words
data_words_nostops = remove_stopwords(data_words)

#getting bigrams
data_words_bigrams = make_bigrams(data_words_nostops)


nlp = spacy.load('en', disable=['parser', 'ner'])

data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['brother', 'law', 'pass']]


In [15]:
id2word = corpora.Dictionary(data_lemmatized)

texts = data_lemmatized

corpus = [id2word.doc2bow(text) for text in texts]

print(corpus[:1])

[[(0, 1), (1, 1), (2, 1)]]


In [16]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_of_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=30,
                                           alpha='auto',
                                           per_word_topics=False)

In [17]:
num_of_words_per_topic = 15
topics = lda_model.print_topics(num_of_topics, num_of_words_per_topic)
doc_lda = lda_model[corpus]
pprint(topics)

[(0,
  '0.033*"call" + 0.025*"feel" + 0.011*"even" + 0.011*"answer" + 0.011*"old" + '
  '0.011*"time" + 0.011*"pain" + 0.011*"mother" + 0.011*"feeling" + '
  '0.011*"daughter" + 0.011*"okay" + 0.011*"need" + 0.011*"shower" + '
  '0.011*"mix" + 0.011*"hang"'),
 (1,
  '0.245*"nervous" + 0.019*"crash" + 0.019*"truck" + 0.019*"tall" + '
  '0.019*"call" + 0.018*"fall" + 0.017*"climb" + 0.010*"see" + 0.010*"tree" + '
  '0.010*"lose" + 0.010*"driver" + 0.010*"die" + 0.010*"crossing" + '
  '0.010*"cross" + 0.010*"next"'),
 (2,
  '0.045*"start" + 0.030*"old" + 0.025*"remember" + 0.015*"drown" + '
  '0.015*"free" + 0.015*"nice" + 0.015*"neighbor" + 0.015*"plastic" + '
  '0.015*"good" + 0.015*"man" + 0.015*"linen" + 0.015*"jumped" + 0.015*"freak" '
  '+ 0.015*"fish" + 0.015*"paz"'),
 (3,
  '0.028*"speak" + 0.028*"large" + 0.028*"front" + 0.026*"group" + '
  '0.014*"show" + 0.014*"impressed" + 0.014*"perfect" + 0.014*"supermarket" + '
  '0.014*"date" + 0.014*"clothe" + 0.014*"cashier" + 0.014*"bea

  '+ 0.017*"procreat" + 0.017*"release" + 0.017*"cementary" + '
  '0.017*"disbelief" + 0.017*"ability" + 0.017*"event" + 0.017*"member" + '
  '0.017*"part" + 0.017*"mind" + 0.017*"calculation"'),
 (30,
  '0.123*"ease" + 0.123*"coworker" + 0.044*"see" + 0.030*"get" + '
  '0.019*"something" + 0.019*"never" + 0.019*"small" + 0.019*"walk" + '
  '0.019*"work" + 0.015*"take" + 0.014*"place" + 0.012*"go" + 0.010*"much" + '
  '0.010*"incredible" + 0.008*"know"'),
 (31,
  '0.312*"would" + 0.051*"think" + 0.030*"day" + 0.023*"work" + 0.021*"happen" '
  '+ 0.020*"not" + 0.011*"take" + 0.011*"never" + 0.011*"know" + '
  '0.011*"friend" + 0.011*"wish" + 0.010*"thing" + 0.010*"be" + 0.010*"bad" + '
  '0.010*"tomorrow"'),
 (32,
  '0.179*"ago" + 0.103*"year" + 0.088*"water" + 0.086*"find" + '
  '0.077*"approximately" + 0.074*"work" + 0.074*"result" + 0.071*"together" + '
  '0.038*"month" + 0.008*"decide" + 0.006*"place" + 0.006*"go" + '
  '0.006*"reservoir" + 0.004*"family" + 0.003*"back"'),
 (33,
  '

In [18]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.453605268650415


In [19]:
mallet_path = 'C:/mallet-2.0.8/mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_of_topics, id2word=id2word)

In [20]:
# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)


Coherence Score:  0.6860686159658128


In [21]:
topics = ldamallet.print_topics(num_of_topics, num_of_words_per_topic)
pprint(topics)

[(0,
  '0.261*"cat" + 0.152*"woman" + 0.109*"call" + 0.087*"person" + 0.043*"move" '
  '+ 0.043*"man" + 0.043*"breathe" + 0.022*"italy" + 0.022*"swimmer" + '
  '0.022*"feeling" + 0.022*"obligate" + 0.022*"fall" + 0.022*"time" + '
  '0.022*"desperately" + 0.022*"santiago"'),
 (1,
  '0.108*"wait" + 0.081*"back" + 0.054*"island" + 0.054*"truck" + 0.054*"trip" '
  '+ 0.027*"week" + 0.027*"presence" + 0.027*"park" + 0.027*"inform" + '
  '0.027*"breakfast" + 0.027*"amazing" + 0.027*"body" + 0.027*"heart" + '
  '0.027*"math" + 0.027*"working"'),
 (2,
  '0.147*"find" + 0.059*"boyfriend" + 0.059*"month" + 0.059*"attention" + '
  '0.029*"chest" + 0.029*"awe" + 0.029*"unrecognizable" + 0.029*"stunning" + '
  '0.029*"pregnant" + 0.029*"jolly" + 0.029*"calm" + 0.029*"floor" + '
  '0.029*"incredible" + 0.029*"content" + 0.029*"cmmunity"'),
 (3,
  '0.172*"walk" + 0.103*"travel" + 0.103*"crash" + 0.103*"scar" + '
  '0.069*"explore" + 0.034*"safety" + 0.034*"bed" + 0.034*"dull" + '
  '0.034*"europe" + 

  '0.029*"work" + 0.029*"hold" + 0.029*"impress" + 0.029*"door" + 0.029*"good" '
  '+ 0.029*"surprise" + 0.029*"ease"'),
 (29,
  '0.182*"occur" + 0.121*"family" + 0.091*"fall" + 0.091*"leave" + '
  '0.061*"february" + 0.061*"eat" + 0.030*"goint" + 0.030*"flood" + '
  '0.030*"kitty" + 0.030*"cashier" + 0.030*"desperately" + 0.030*"sector" + '
  '0.030*"astonishment" + 0.030*"retreat" + 0.030*"full"'),
 (30,
  '0.267*"life" + 0.200*"time" + 0.133*"find" + 0.100*"world" + 0.033*"labor" '
  '+ 0.033*"spasm" + 0.033*"cup" + 0.033*"tire" + 0.033*"romantic" + '
  '0.033*"miss" + 0.033*"shower" + 0.033*"astonishing" + 0.033*"summit" + '
  '0.000*"televison" + 0.000*"inmense"'),
 (31,
  '0.128*"baby" + 0.085*"decide" + 0.064*"pain" + 0.064*"good" + '
  '0.064*"sensation" + 0.064*"thinking" + 0.043*"fourth" + 0.043*"work" + '
  '0.043*"test" + 0.043*"reservoir" + 0.021*"answer" + 0.021*"week" + '
  '0.021*"prior" + 0.021*"plan" + 0.021*"laugh"'),
 (32,
  '0.079*"group" + 0.053*"nervous" + 0.053*

In [22]:
index = 0
chunks = [None] * num_of_topics
allKeywords = [None] * num_of_topics
for chunk in topics:
    chunk = chunk[1]
    percentages = re.findall(r"[-+]?\d*\.\d+|\d+", chunk) #credit to miku on Stackoverflow
    keywords = re.findall('"([^"]*)"', chunk) #credit to jspcal on Stackoverflow
    allKeywords[index] = keywords
    result = [None] * 2 * num_of_words_per_topic
    result[::2] = percentages
    result[1::2] = keywords
    result = [str(index)] + result
    chunks[index] = result # A array stroing arrays of keywords and corresponding percentages
    index += 1
print(chunks)
    


[['0', '0.261', 'cat', '0.152', 'woman', '0.109', 'call', '0.087', 'person', '0.043', 'move', '0.043', 'man', '0.043', 'breathe', '0.022', 'italy', '0.022', 'swimmer', '0.022', 'feeling', '0.022', 'obligate', '0.022', 'fall', '0.022', 'time', '0.022', 'desperately', '0.022', 'santiago'], ['1', '0.108', 'wait', '0.081', 'back', '0.054', 'island', '0.054', 'truck', '0.054', 'trip', '0.027', 'week', '0.027', 'presence', '0.027', 'park', '0.027', 'inform', '0.027', 'breakfast', '0.027', 'amazing', '0.027', 'body', '0.027', 'heart', '0.027', 'math', '0.027', 'working'], ['2', '0.147', 'find', '0.059', 'boyfriend', '0.059', 'month', '0.059', 'attention', '0.029', 'chest', '0.029', 'awe', '0.029', 'unrecognizable', '0.029', 'stunning', '0.029', 'pregnant', '0.029', 'jolly', '0.029', 'calm', '0.029', 'floor', '0.029', 'incredible', '0.029', 'content', '0.029', 'cmmunity'], ['3', '0.172', 'walk', '0.103', 'travel', '0.103', 'crash', '0.103', 'scar', '0.069', 'explore', '0.034', 'safety', '0.034

In [23]:
print(allKeywords)

[['cat', 'woman', 'call', 'person', 'move', 'man', 'breathe', 'italy', 'swimmer', 'feeling', 'obligate', 'fall', 'time', 'desperately', 'santiago'], ['wait', 'back', 'island', 'truck', 'trip', 'week', 'presence', 'park', 'inform', 'breakfast', 'amazing', 'body', 'heart', 'math', 'working'], ['find', 'boyfriend', 'month', 'attention', 'chest', 'awe', 'unrecognizable', 'stunning', 'pregnant', 'jolly', 'calm', 'floor', 'incredible', 'content', 'cmmunity'], ['walk', 'travel', 'crash', 'scar', 'explore', 'safety', 'bed', 'dull', 'europe', 'april', 'impotent', 'waist', 'side', 'cup', 'huddle'], ['son', 'ago', 'planet', 'drop', 'member', 'talcahuano', 'inexplicable', 'calculation', 'section', 'work', 'remove', 'concentrate', 'clinic', 'freeze', 'stomach'], ['happen', 'people', 'pale', 'situation', 'promote', 'rob', 'hang', 'diagnose', 'culture', 'organization', 'move', 'instantenously', 'compile', 'clinic', 'opportunity'], ['chile', 'fear', 'lake', 'enjoy', 'kiss', 'vacation', 'arrive', 'heli

In [24]:
header = [None] * 2 * num_of_words_per_topic

for i in range(0, 2 * num_of_words_per_topic):
    if i % 2 == 0:
               header[i] = 'Percentage'
    else:
               header[i] = 'Keyword'

header = ['Topic No.'] + header
topicsPath = outputPath + dataName + '_' + 'topics.csv'
with open(topicsPath, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows([header])
    for chunk in chunks:
        writer.writerows([chunk])

In [26]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    sent_topics_df = pd.DataFrame()

    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)

        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamallet, corpus=corpus, texts=data)

df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Index', 'Dominant Topic', 'Topic Contribution', 'Keywords', 'Text']

dominantTopicsPath = outputPath + dataName + '_' + 'dominant_topic.csv'
df_dominant_topic.to_csv(dominantTopicsPath)

In [27]:
# For generating top 'num_of_representatives' most representative narratives


import operator
repre = [None] * 50
for i, row in enumerate(ldamallet[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(row):
            if repre[topic_num] is None:
                repre[topic_num] = dict()
            repre[topic_num][str(i)] = prop_topic
sorted_repre = [None] * 50
for i in range(0, len(repre)):
    sorted_repre[i] = sorted(repre[i].items(), key=operator.itemgetter(1), reverse = True)

#pprint(sorted_repre)
allTheTopics = [None] * 50
index = 0
for item in sorted_repre:
    allTheTopics[index] = [index]
    allTheTopics[index] += [allKeywords[index]]
    for i in range(0, num_of_representatives):
        allTheTopics[index] += [data[int(sorted_repre[index][i][0])]]
    index += 1
print(allTheTopics)
representativeNarraPath = outputPath + dataName + '_' + 'representative_narratives.csv'
header = (2 + num_of_representatives) * [None]
header[0] = 'Topic No.'
header[1] = 'Keywords'
for i in range(1, num_of_representatives+1):
    header[i + 1] = i
with open(representativeNarraPath, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows([header])
    for topic in allTheTopics:
        writer.writerows([topic])

[[0, ['cat', 'woman', 'call', 'person', 'move', 'man', 'breathe', 'italy', 'swimmer', 'feeling', 'obligate', 'fall', 'time', 'desperately', 'santiago'], 'A few years ago I was walking through a familiar place in Santiago, Chile, Italy plaza when I saw many people gathered watching an enormous cat who was trying to come down from a tall palm tree. The cat screamed intensely and snored desperately. A woman and two men arrived with a metal ladder but it was not tall enough. The woman climbed the ladder and tried calling the cat, the ladder would move and she persisted drawing the cats attention with a branch. The cat, scared, reacted and lost balance falling into nothing but the woman dropped the branch and was able to catch the cat, leaving the ladder unbalanced. When the woman climbed down huddling the cat, it made me realize I should do more for abandoned animals and wished to have been the person with so much courage.', 'My cat was pregnant and I was alone at home, I was not aware it 

In [28]:
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

sent_topics_outdf_grpd.head()

,Dominant_Topic,Perc_Contribution,Topic_Keywords,0
0,47.0,0.0356,"brother, remember, car, safe, completely, swim...",When my brother-in-law passed away
1,19.0,0.0331,"moment, discover, feel, change, surprise, unde...","When I changed jobs, I thought it was for the ..."
2,34.0,0.0363,"bear, give, fall, arrive, earth, flee, prize, ...",When my son was born
3,36.0,0.0601,"cry, south, familiar, immediately, fire, turn,...",When my partner told me that they got fired fr...
4,1.0,0.0496,"wait, back, island, truck, trip, breakfast, am...",My kitten disappeared for two weeks and we fin...
5,42.0,0.0741,"thing, front, sleep, watch, leave, lagoon, pre...",It started when I had speak in front of a larg...
6,31.0,0.0732,"baby, decide, pain, good, sensation, thinking,...",I was in awe when my first daughter was born b...
7,15.0,0.0417,"earthquake, year, house, return, flee, occur, ...",After the earthquake in Chile in 2010/ Santiag...
8,28.0,0.0448,"house, due, movie, strong, closer, arthritis, ...","kissing, it was in the downtown antofagasta, w..."
9,38.0,0.0326,"mom, father, money, work, bad, inability, abil...","it was a strong earthquake, with risk fo a tsu..."


In [30]:

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)
   
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

sent_topics_sorteddf_mallet.columns = ['Topic Index', "Topic Contribution", "Keywords", "Text"]

sent_topics_sorteddf_mallet.head(10)

ValueError: Length mismatch: Expected axis has 8 elements, new values have 4 elements